In [3]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import pandas as pd
import os
os.environ["TOKENIZERS_PARALLELISM"] = "1"

from config import Config
from fliker_comment_tokenizer import FlikerCommentTokenizer
from pathlib import Path
from transformers import AutoTokenizer
from tqdm import tqdm


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = Config()

fliker_comment_toeknizer = FlikerCommentTokenizer.train_tokenizer(config=config)
fliker_comment_toeknizer

Enriched img id: /tmp/enriched_results.csv


100%|██████████| 159/159 [00:00<00:00, 565.35it/s]





Saved new fliker comment tokenizer at: /Users/chengbai/ml/cheng_git/notebooks/paligemma-3b-mix-224-tokenizer


GemmaTokenizerFast(name_or_path='google/paligemma-3b-mix-224', vocab_size=10000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<image>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [3]:
new_fliker_comment_toeknizer = FlikerCommentTokenizer.get_tokenizer(config=config)

# test_data = "hello, 會 員"
test_data = "<image><bos>cheng bai<pad><eos>"
test_data_encoded = new_fliker_comment_toeknizer.encode(test_data)
print(f"encoded: {test_data_encoded}")
test_data_decoded = new_fliker_comment_toeknizer.decode(test_data_encoded)
print(f"decode: {test_data_decoded}")

tokens: 10000
tokenizer.is_fast: True
encoded: [2, 4, 2, 6473, 144, 183, 68, 0, 1]
decode: <bos><image><bos>cheng bai<pad><eos>


In [4]:
print(f"<image>: {new_fliker_comment_toeknizer.encode('<image>')}")
print(f"<bos>: {new_fliker_comment_toeknizer.encode('<bos>')}")
print(f"<eos>: {new_fliker_comment_toeknizer.encode('<eos>')}")
print(f"<pad>: {new_fliker_comment_toeknizer.encode('<pad>')}")
print(f"cheng bai: {new_fliker_comment_toeknizer.encode('cheng bai')}")

<image>: [2, 4]
<bos>: [2, 2]
<eos>: [2, 1]
<pad>: [2, 0]
cheng bai: [2, 6473, 144, 183, 68]


In [ ]:
new_fliker_comment_toeknizer.decode(0)

In [ ]:
dir(new_fliker_comment_toeknizer)

In [ ]:
new_fliker_comment_toeknizer.vocab_size, len(new_fliker_comment_toeknizer.get_vocab()),
new_fliker_comment_toeknizer.get_added_vocab()

for k, v in new_fliker_comment_toeknizer.vocab.items():
    if v == 0:
        print(k)
        break

In [40]:
target = torch.eye(2, dtype=torch.float32)  # 64 classes, batch size = 10
output = torch.eye(2, dtype=torch.float32) * 10000  # 64 classes, batch size = 10
# output = torch.arange(4, dtype=torch.float32).view(2, -1)  # A prediction (logit)
pos_weight = torch.ones([2])  # All weights are equal to 1
# criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion = torch.nn.BCEWithLogitsLoss()
criterion(output, target)  # -log(sigmoid(1.5))

tensor(0.3466)

In [41]:
pos_weight

tensor([1., 1.])

In [42]:
output

tensor([[10000.,     0.],
        [    0., 10000.]])

In [43]:
target

tensor([[1., 0.],
        [0., 1.]])

In [44]:
sig_output = nn.Sigmoid()(output)
sig_output

tensor([[1.0000, 0.5000],
        [0.5000, 1.0000]])

In [36]:
(
    1 * sig_output[0][0].log()
    + (1 - sig_output[0][1]).log()
    + (1 - sig_output[1][0]).log()
    + sig_output[1][1].log()
) / 4

tensor(-0.5032)